# Neural Style Transfer


This tutorial uses deep learning to compose one image in the style of another image (ever wish you could paint like Picasso or Van Gogh?). This is known as *neural style transfer* and the technique is outlined in <a href="https://arxiv.org/abs/1508.06576" class="external">A Neural Algorithm of Artistic Style</a> by Gatys et al. 


Neural style transfer is an optimization technique that takes input two images — a *content* image and a *style reference* image (such as an artwork by a famous painter)—and blend them together so the output image looks like the content image, but “painted” in the style of the style reference image.

This is implemented by optimizing the output image to match the content statistics of the content image and the style statistics of the style reference image. These statistics are extracted from the images using a convolutional neural network as depicted below.




![Overview of Neural Style Transfer](https://drive.google.com/uc?id=1YIJaRqS4a_gFHuyrGBQgBHcmXc2hnsAW)

## Setup


### Import and configure modules

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

Complete the code

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


TensorFlow 2.x selected.


In [0]:
# IMPORT Tensorflow here
#YOUR CODE HERE 

#YOUR CODE ENDS HERE 

##Capture Photo to be used 


In [0]:
# Use "Camera Capture" from code snippets of colab

Download images and choose a style image and a content image:

In [0]:
#Add your content path here
content_path = # YOUR CODE HERE

# In Case Your webcam isn't good, uncommment and  paste your desired image url here!
# content_path = tf.keras.utils.get_file('78686742_1106700599500165_457257697394294784_o.jpg','https://scontent.fdel29-1.fna.fbcdn.net/v/t1.0-9/p960x960/78686742_1106700599500165_457257697394294784_o.jpg?_nc_cat=102&_nc_ohc=zv7FeBJ8zkYAX9q6bEN&_nc_ht=scontent.fdel29-1.fna&_nc_tp=6&oh=307311755eda6eea99d02f1b05143bf6&oe=5E8EE134')


style_path = tf.keras.utils.get_file('starry.jpg','https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT-YQpREaV8eBEIjkoG3gKa8Q9T_8OvRy4d2z_I7YI-zhjI_7x-')

SyntaxError: ignored

In [0]:
import IPython.display as display
import time
import functools

In [0]:
# IMPORT matplotlib, pyplot , numpy and PIL.Image here
# YOUR CODE HERE

# YOUR CODE ENDS HERE 

In [0]:
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

In [0]:
def tensor_to_image(tensor): # just a helper function to convert the inputs/outputs to displayable format
  # YOUR CODE HERE 

  # YOUR CODE ENDS HERE 
  # tensors are in range 0-1 ....rescale them back to 0-255 and then type cast into integer format  for RGB images
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

## Visualize the input

Define a function to load an image and limit its maximum dimension to 512 pixels.

In [0]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32) # just some tensorflow functions to read jpeg images and convert them to usable format

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  print(shape)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)
  #Complete code to reshape the image to shape as defined in max_dim
  # YOUR CODE HERE 
                

  #YOUR CODE ENDS HERE 
  return img

Create a simple function to display an image:

In [0]:
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0) # remove the extra batch dimension to display a single image 

  plt.imshow(image)
  if title:
    plt.title(title)

In [0]:
# load and display content and style image 
content_image = # YOUR CODE HERE
style_image   =  # YOUR CODE HERE

# Complete code to display image
# YOUR CODE HERE


# YOUR CODE HERE

## Define content and style representations

Use the intermediate layers of the model to get the *content* and *style* representations of the image. Starting from the network's input layer, the first few layer activations represent low-level features like edges and textures. As you step through the network, the final few layers represent higher-level features—object parts like *wheels* or *eyes*. In this case, you are using the VGG19 network architecture, a pretrained image classification network. These intermediate layers are necessary to define the representation of content and style from the images. For an input image, try to match the corresponding style and content target representations at these intermediate layers.


Now load a `VGG19` without the classification head, and list the layer names

In [0]:
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

print()
for layer in vgg.layers:
  print(layer.name)
vgg.summary()

Choose intermediate layers from the network to represent the style and content of the image.

Style -> block5_conv2

Content -> block1_conv1, block2_conv1, block3_conv1,   block4_conv1, block5_conv1

In [0]:
# define style and content layers
# YOUR CODE HERE
content_layers =

style_layers   = 
# YOUR CODE ENDS HERE

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

#### Intermediate layers for style and content

So why do these intermediate outputs within our pretrained image classification network allow us to define style and content representations?

At a high level, in order for a network to perform image classification (which this network has been trained to do), it must understand the image. This requires taking the raw image as input pixels and building an internal representation that converts the raw image pixels into a complex understanding of the features present within the image.

This is also a reason why convolutional neural networks are able to generalize well: they’re able to capture the invariances and defining features within classes (e.g. cats vs. dogs) that are agnostic to background noise and other nuisances. Thus, somewhere between where the raw image is fed into the model and the output classification label, the model serves as a complex feature extractor. By accessing intermediate layers of the model, you're able to describe the content and style of input images.

## Build the model 

The networks in `tf.keras.applications` are designed so you can easily extract the intermediate layer values using the Keras functional API.

To define a model using the functional API, specify the inputs and outputs:

`model = Model(inputs, outputs)`

This following function builds a VGG19 model that returns a list of intermediate layer outputs:

In [0]:
def vgg_layers(layer_names):
  """ Creates a vgg model that returns a list of intermediate output values."""
  # Load our model. Load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  
  # returns the outputs of all the intermediate layers having names in layer_names
  # YOUR CODE HERE 
                                      
  # YOUR CODE ENDS HERE 
  
  # create a new  model  specifying the inputs and the outputs from the layers you want
  # YOUR CODE HERE 
  
  #YOUR CODE ENDS HERE
  return model

## Calculate style

The content of an image is represented by the values of the intermediate feature maps.

It turns out, the style of an image can be described by the means and correlations across the different feature maps. Calculate a Gram matrix that includes this information by taking the outer product of the feature vector with itself at each location, and averaging that outer product over all locations. This Gram matrix can be calcualted for a particular layer as:

$$G^l_{cd} = \frac{\sum_{ij} F^l_{ijc}(x)F^l_{ijd}(x)}{IJ}$$

This can be implemented concisely using the `tf.linalg.einsum` function:

In [0]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)

## Extract style and content


Build a model that returns the style and content tensors.

In [0]:
class StyleContentModel(tf.keras.models.Model):
  def __init__(self, style_layers, content_layers): # this is basically the constructor of the class/object which holds all the layers and other variables of the model
    super(StyleContentModel, self).__init__()
    self.vgg =  vgg_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.vgg.trainable = False # We don't want to change the model  , we just want to tweak the input image

  def call(self, inputs):
    "Expects float input in [0,1]"
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
     
    #Compute  style and content output 
    # YOUR CODE HERE 

    # YOUR CODE ENDS HERE 

    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)} # store the results in a dictionary and return it 
    
    return {'content':content_dict, 'style':style_dict}

When called on an image, this model returns the gram matrix (style) of the `style_layers` and content of the `content_layers`:

In [0]:
# run the model on the style and content images
#YOUR CODE HERE 
extractor =  
#YOUR CODE ENDS HERE 

## Let's Visualize the results of those convolutions layers!

In [0]:
import numpy as np
import random
from   tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.

visualization_model = extractor.vgg

img = load_img(content_path, target_size=(512, 512))  # this is a PIL image

x   = img_to_array(img)                           # Numpy array with shape (512, 512, 3)
x   = x.reshape((1,) + x.shape)                   # Numpy array with shape (1, 512, 512, 3)

# Rescale by 1/255
x /= 255.0

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in visualization_model.layers]

# -----------------------------------------------------------------------
# Now let's display our representations
# -----------------------------------------------------------------------
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  
  if len(feature_map.shape) == 4:
    
    #-------------------------------------------
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    #-------------------------------------------
    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
    
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    
    #-------------------------------------------------
    # Postprocess the feature to be visually palatable
    #-------------------------------------------------
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x # Tile each filter into a horizontal grid

    #-----------------
    # Display the grid
    #-----------------

    scale = 200. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' ) 

## Run gradient descent

With this style and content extractor, you can now implement the style transfer algorithm. Do this by calculating the mean square error for your image's output relative to each target, then take the weighted sum of these losses.

Set your style and content target values:

In [0]:
# get the style and content target values

# YOUR CODE HERE
style_targets = 
content_targets =
# YOUR CODE ENDS HERE  

Define a `tf.Variable` to contain the image to optimize. To make this quick, initialize it with the content image (the `tf.Variable` must be the same shape as the content image):

In [0]:
image = tf.Variable(content_image) # tf.Variable is used for variables we require to tweak via backpropagation

Since this is a float image, define a function to keep the pixel values between 0 and 1:

In [0]:
def clip_0_1(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

Create an optimizer. The paper recommends LBFGS, but `Adam` works okay, too:

In [0]:
# define optimizer
opt =  # YOUR CODE HERE

To optimize this, use a weighted combination of the two losses to get the total loss:

In [0]:
style_weight   = 1e-2
content_weight = 1e4

In [0]:
def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']

    #YOUR CODE HERE
    style_loss  = 
    style_loss *= style_weight / num_style_layers

    content_loss = 
    content_loss *= content_weight / num_content_layers
    #YOUR CODE ENDS HERE 

    loss = style_loss + content_loss
    return loss

In [0]:
total_variation_weight = 30

Use `tf.GradientTape` to update the image.

In [0]:
@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)  #pass image through model
    loss = style_content_loss(outputs)  #calulate style and content loss
    loss += total_variation_weight*tf.image.total_variation(image) # add total variation loss

#YOUR CODE HERE 
  grad =               # caclulate gradient of loss wrt image
                       # apply gradient to perform optimization step

#YOUR CODE ENDS HERE 
  image.assign(clip_0_1(image))

Now run a few steps to test:

In [0]:
train_step(image)
tensor_to_image(image)
train_step(image)
tensor_to_image(image)
train_step(image)
tensor_to_image(image)

And run the optimization:

In [0]:
import time
start = time.time()

#YOUR CODE HERE
epochs =          # define number of epochs and steps per epoch
steps_per_epoch = 
#YOUR CODE ENDS HERE 


step = 0
for n in range(epochs):
  for m in range(steps_per_epoch): # Basically run the algorithm for multiple steps to get better results
    step += 1
    # ADD training step here
    #YOUR CODE HERE
    
    #YOUR CODE ENDS HERE 
    print(".", end='') 
  display.clear_output(wait=True)
  display.display(tensor_to_image(image))
  print("Train step: {}".format(step))

end = time.time()
print("Total time: {:.1f}".format(end-start))

Finally, save the result:

In [0]:
file_name = 'stylized-image.png'
tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

## Fast Style Transfer using TF-Hub

This tutorial demonstrates the original style-transfer algorithm. Which optimizes the image content to a particular style. Before getting into the details let's see how the [TensorFlow Hub](https://tensorflow.org/hub) module does:

In [0]:
import tensorflow_hub as hub
hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/1')
stylized_image = hub_module(tf.constant(content_image), tf.constant(style_image))[0]
tensor_to_image(stylized_image)